In [1]:
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql.types import *
from functools import reduce
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as f
import time

import pandas as pd
import numpy as np

In [2]:
sc = SparkContext('local')
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

In [3]:
csv_list = ["09","10","11","12"]
df_list = []
activityList = [1001,1003,1004,1007,1008,1010,1099,1101,1102,1104,1105,1199,1201,1202,1203,1204,1205,1206,1207,1208,1299,2002,2003,2004,2099,2104,2107,2109,2110,2111,2112,2113,2199,2401,2402,2403,2404,2406,2407,2499,2299,2312,2317,4113,7104,7299,6001,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011,6012,6013,6015,6016,6021,6099,6014,6019,6020,6018,6107,6113,6201,6202,6203,6204,6205,6206,6207,6208,6209,6210,6111,6112,6299,7101,7102,7103,7105,7106,7108,7199]

for listname in csv_list:
    df = sqlContext.read.format('com.databricks.spark.csv')\
        .options(header='true', inferSchema='true')\
        .load('C:/ssafy/2nd/s03p23a305/bigdata/analysis/2019-'+listname+'-UTF.csv')\
        .cache()
    
    # 기업 데이터 제거
    df = df.filter(df.개인기업구분 != "기업")
    df = df.filter(df.성별 != "0.기타")
    
    # 활동 분류
    df = df.filter(df.가맹점업종코드.isin(activityList))
    
    removeColList = ["회원_시도명", "회원_시군구명", "개인기업구분", "매출년월일", "결제금액", "결제건수"]
    for ls in removeColList:
        df = df.drop(ls)
    df.persist()
    df_list.append(df)

AnalysisException: Path does not exist: file:/C:/ssafy/2nd/s03p23a305/bigdata/analysis/2019-09-UTF.csv;

start = time.time()  # 시작 시간 저장

df_list[0].show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
# schema = StructType([StructField("성별", StringType(), True), StructField("연령", StringType(), False),\
#                      StructField("승인시간대1", IntegerType(), True), StructField("가맹점업종코드", IntegerType(), True)])

# df_all = sqlContext.createDataFrame([],schema)
df_all = df_list[0]
# df_all.show()

In [ ]:
for idx, df in enumerate(df_list):
    if(idx == 0):
        continue
    print(idx)
    df_all = df_all.union(df)

df_all.show()

In [ ]:
df_all.colRegex

In [ ]:
df_all_renamed = df_all.withColumnRenamed("성별", "gender")\
                        .withColumnRenamed("연령", "age")\
                        .withColumnRenamed("승인시간대1", "time")\
                        .withColumnRenamed("가맹점_시도명", "do")\
                        .withColumnRenamed("가맹점_시군구명", "si")\
                        .withColumnRenamed("가맹점_읍면동명", "dong")\
                        .withColumnRenamed("가맹점업종코드", "code")\
                        .withColumnRenamed("회원수", "n")                        

df_all_renamed.show()

In [ ]:
rdd = df_all_renamed.rdd
rdd

In [ ]:
rdd.take(3)

In [ ]:
rdd = rdd.map(lambda x: Row(gender=int(x[0][0]),\
                            age=int(x[1][0]),\
                            time=int(x[2]),\
                            do=x[3],\
                            si=x[4],\
                            dong = x[5],\
                            code =int(x[6]),\
                            n = int(x[7])))

In [ ]:
rdd.take(3)[0]

type(rdd)

In [ ]:
schema = StructType([StructField("gender", StringType(), True),\
                     StructField("age", StringType(), True),\
                     StructField("time", IntegerType(), True),\
                     StructField("do", StringType(), True),\
                     StructField("si", StringType(), True),\
                     StructField("dong", StringType(), True),\
                     StructField("code", IntegerType(), True),\
                     StructField("n", IntegerType(), True)])


In [ ]:
# df_mapped = spark.createDataFrame(rdd, schema).cache()
df_mapped = spark.createDataFrame(rdd, schema).persist()

# 캐싱 없이 쇼
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

# 캐싱 없이 재차 쇼
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

# 캐싱 후 쇼
df_mapped.persist()
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

df_mapped.unpersist()

In [ ]:
start = time.time()  # 시작 시간 저장

df_mapped.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
df_mapped.registerTempTable("df_tmp")

df_mapped.count()

In [ ]:
# 코드 형식
# df_mapped.groupBy('gender', 'age','time', 'code').count().select('gender','age','time', f.col('count').alias('n')).show()

# df_mapped.groupBy('gender', 'age','time', 'code').sum('n').show() # count().select('gender','age','time', f.col('count').alias('n')).show()
df_group = df_mapped.groupBy('gender', 'age','time', 'do', 'si','dong', 'code').sum('n')

# 쿼리 형식
df_group = sqlContext.sql("select *, sum(n) as cnt from df_tmp group by gender, age, time, code")

In [ ]:
df_group = df_group.withColumnRenamed("sum(n)", "total").persist()
df_group.columns

In [ ]:
start = time.time()  # 시작 시간 저장

df_group.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

In [19]:
df_group.count()

1163345

In [19]:
df_group.registerTempTable("df_group")

In [20]:
dong = "'영등포동'"


In [21]:
query = "select gender, age, time, code, total from df_group where dong ="+ dong +" order by total desc"
query

"select gender, age, time, code, total from df_group where dong ='영등포동' order by total desc"

In [22]:
selected_df = sqlContext.sql(query).persist()

# selected_df.show()

selected_df.persist()

In [23]:
selected_df.columns

['gender', 'age', 'time', 'code', 'total']

selected_df.describe()

selected_df.show()

In [24]:
selected_df.registerTempTable("selected_df")

In [25]:
query = "select code, sum(total) as sum from selected_df group by code order by sum desc"
query

'select code, sum(total) as sum from selected_df group by code order by sum desc'

In [26]:
sum_groupByDf = sqlContext.sql(query).persist()


In [27]:
start = time.time()  # 시작 시간 저장

sum_groupByDf.show()

print("time :", time.time() - start)  # 현재시각 - 시작시간 = 실행 시간

+----+----+
|code| sum|
+----+----+
|2099|7240|
|2004|5873|
|2003|5731|
|2199|4043|
|6014|3107|
|2104|3013|
|2002|1380|
|7106|1047|
|1004| 989|
|2406| 211|
|4113|  69|
|2402|  65|
|2112|  52|
|7103|  21|
|7102|  15|
|2109|   3|
+----+----+

time : 17.43963932991028


In [68]:
total_sum = 0
for row in selected_df.collect():
    total_sum += row.total
total_sum

1964088

In [ ]:
michelin = spark.read.json('C:/ssafy/2nd/s03p22a305/bigdata/analysis/data_data_josn')
michelin

In [107]:
code_df = sqlContext.read.format('com.databricks.spark.csv')\
        .options(header='true', inferSchema='true')\
        .load('C:/ssafy/2nd/s03p22a305/bigdata/analysis/codelist-UTF.csv')\
        .cache()

In [118]:
code_df.show()

+--------+-----------+------------------+----+----------------------------------+
|업종분류|       업종|                업|코드|                  적용범위 및 기준|
+--------+-----------+------------------+----+----------------------------------+
|  의생활|       의류|          맞춤복점|1001|남성복 맞춤업소,여성복 맞춤업소...|
|  의생활|       의류|            한복점|1003|  남.여 한복 맞춤업소, 개량한복...|
|  의생활|       의류|          기성복점|1004|  의류대리점(모피 및 교복 포함)...|
|  의생활|       의류|  아동 및 유아복점|1007|           아동복 및 유아복 판매점|
|  의생활|       의류|        내의판매점|1008|    내의,타월,양말,스타킹등 판매점|
|  의생활|       의류|            양품점|1010|             각종 양품,잡화 판매점|
|  의생활|       의류|         기타 의류|1099|                              null|
|  의생활|직물/침구류|        옷감판매점|1101|  양복,양장,한복등의 옷감 및 면...|
|  의생활|직물/침구류|침구,커튼,카펫트점|1102|  이불,담요,베개,카바,커튼,카펫...|
|  의생활|직물/침구류|            수예점|1104|   각종 수예품(자수,실포함) 판매점|
|  의생활|직물/침구류|            자석요|1105|              자석요 수입,제조판매|
|  의생활|직물/침구류|         기타 직물|1199|                              null|
|  의생활|   신변잡화|        악세사리점|1